In [8]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [14]:
action = ['applauding','blowing bubbles','brushing teeth','cleaning the floor','cleaning the floor','climbing','cooking','cutting trees','cutting vegetables','drinking','feeding a horse','fishing','fixing a bike','fixing a car','gardening','holding an umbrella','jumping','looking through a microscope','looking through a telescope','phoning','playing guitar','playing violin','pouring liquid','pushing a cart','reading','riding a bike','riding a horse','rowing a boat','running','shooting an arrow','smoking','taking photos','texting message','throwing frisby','using a computer','walking the dog','washing dishes','watching TV','waving hands','writing on a board','writing on a book']

import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

df=pd.DataFrame()
y=pd.DataFrame(columns=['action'])

def clamp(n, minn, maxn): 
    return max(min(maxn, n), minn)

# loop through every image

for character in action:
    for image in os.listdir('./Action recog/Images/'):
        # read image as grayscale
        img = cv2.imread('./Action recog/Images/' + image , 0)
        imgOG = cv2.resize(img, (256, 256))
        #flatten image
        imgFlat = imgOG.flatten()
        # add image to dataset
        df = pd.concat([df, pd.DataFrame(imgFlat).T], ignore_index=True)
        # add label to y
        y = pd.concat([y, pd.DataFrame([character], columns=['action'])], ignore_index=True)
df

KeyboardInterrupt: 

In [ ]:
y

In [ ]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
# calculate mean of each column
mean = df.mean(axis=0)
#show mean face by unflattening
plt.imshow(mean.values.reshape(256, 256), cmap='gray')
plt.show()

In [ ]:
# standardize data using standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#function to do PCA with given number of components
def doPCA(n):
    from sklearn.decomposition import PCA
    pca = PCA(n_components=n)
    pca.fit(X_train)
    X_train_pca = pca.transform(X_train)
    X_test_pca = pca.transform(X_test)
    return X_train_pca, X_test_pca

# doing pca with 5 components
X_train_pca, X_test_pca = doPCA(5)

# 1-NN classifier
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train_pca, y_train.values.ravel())
print("Accuracy Score is : {:.2f}".format(knn.score(X_test_pca, y_test)))

# predict on test set
y_pred = knn.predict(X_test_pca)

# print confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
# print(cm)

# print classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

# plot confusion matrix
import seaborn as sns
plt.figure(figsize=(5, 5))
sns.heatmap(cm, annot=True, fmt="d")
plt.title("Confusion matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()
